In [1]:
library(tidyverse)
library(repr)
library(digest)
library(gridExtra)
library(cowplot)
library(dplyr)
library(tidymodels)
library(GGally)
library(splines)

Warning message:
“package ‘tidyverse’ was built under R version 4.1.2”
── Attaching packages ─────────────────────────────────────── tidyverse 1.3.2 ──
✔ ggplot2 3.4.0      ✔ purrr   0.3.4 
✔ tibble  3.1.8      ✔ dplyr   1.0.10
✔ tidyr   1.2.0      ✔ stringr 1.4.0 
✔ readr   2.1.3      ✔ forcats 0.5.1 
Warning message:
“package ‘ggplot2’ was built under R version 4.1.2”
Warning message:
“package ‘tibble’ was built under R version 4.1.2”
Warning message:
“package ‘tidyr’ was built under R version 4.1.2”
Warning message:
“package ‘readr’ was built under R version 4.1.2”
Warning message:
“package ‘dplyr’ was built under R version 4.1.2”
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
Warning message:
“package ‘repr’ was built under R version 4.1.2”

Attaching package: ‘gridExtra’


The following object is masked from ‘package:dplyr’:

    combine


Warning message:
“package ‘tidym

In [2]:
data <- read_csv("data/FantasyPointsData.csv")
head(data)

New names:
• `` -> `...12`
Rows: 539 Columns: 12
── Column specification ────────────────────────────────────────────────────────
Delimiter: ","
chr (2): last_name, first_name
dbl (9): player_id, year, b_strikeout, b_walk, b_rbi, b_total_bases, r_total...
lgl (1): ...12

ℹ Use `spec()` to retrieve the full column specification for this data.
ℹ Specify the column types or set `show_col_types = FALSE` to quiet this message.


last_name,first_name,player_id,year,b_strikeout,b_walk,b_rbi,b_total_bases,r_total_stolen_base,b_game,r_run,...12
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<lgl>
Cruz Jr.,Nelson,443558,2022,119,49,64,151,4,124,50,NA
Blackmon,Charlie,453568,2022,109,32,78,222,4,135,60,NA
McCutchen,Andrew,457705,2022,124,57,69,198,8,134,66,NA
Turner,Justin,457759,2022,89,50,81,205,3,128,61,NA
Andrus,Elvis,462101,2022,92,39,58,216,18,149,66,NA
Santana,Carlos,467793,2022,88,71,60,162,0,131,52,NA


In [4]:
selected <- data %>% select(-"player_id", -"...12")
head(selected)

last_name,first_name,year,b_strikeout,b_walk,b_rbi,b_total_bases,r_total_stolen_base,b_game,r_run
<chr>,<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
Cruz Jr.,Nelson,2022,119,49,64,151,4,124,50
Blackmon,Charlie,2022,109,32,78,222,4,135,60
McCutchen,Andrew,2022,124,57,69,198,8,134,66
Turner,Justin,2022,89,50,81,205,3,128,61
Andrus,Elvis,2022,92,39,58,216,18,149,66
Santana,Carlos,2022,88,71,60,162,0,131,52


In [5]:
names <- selected %>% unite("player", last_name:first_name, sep = ", ", remove = T) %>% 
rename("K" = "b_strikeout", "BB" = "b_walk", "RBI" = "b_rbi", "TB" = "b_total_bases", "SB" = "r_total_stolen_base", 
           "R" = "r_run", "G" = "b_game")
head(names)
                                                                                               


player,year,K,BB,RBI,TB,SB,G,R
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Cruz Jr., Nelson",2022,119,49,64,151,4,124,50
"Blackmon, Charlie",2022,109,32,78,222,4,135,60
"McCutchen, Andrew",2022,124,57,69,198,8,134,66
"Turner, Justin",2022,89,50,81,205,3,128,61
"Andrus, Elvis",2022,92,39,58,216,18,149,66
"Santana, Carlos",2022,88,71,60,162,0,131,52


In [6]:
fpoints <- names %>% mutate(Fpoints = BB + RBI + TB + SB + R - K)
head(fpoints)

player,year,K,BB,RBI,TB,SB,G,R,Fpoints
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Cruz Jr., Nelson",2022,119,49,64,151,4,124,50,199
"Blackmon, Charlie",2022,109,32,78,222,4,135,60,287
"McCutchen, Andrew",2022,124,57,69,198,8,134,66,274
"Turner, Justin",2022,89,50,81,205,3,128,61,311
"Andrus, Elvis",2022,92,39,58,216,18,149,66,305
"Santana, Carlos",2022,88,71,60,162,0,131,52,257


In [12]:
fpoints_g <- fpoints %>% mutate(fpoints_g = Fpoints / G) %>% arrange(desc(fpoints_g))
head(fpoints_g)

player,year,K,BB,RBI,TB,SB,G,R,Fpoints,fpoints_g
<chr>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>,<dbl>
"Soto, Juan",2020,28,41,37,107,5,47,39,201,4.276596
"Freeman, Freddie",2020,37,45,53,137,2,60,51,251,4.183333
"Yelich, Christian",2019,118,80,97,328,30,130,100,517,3.976923
"Rendon, Anthony",2019,86,80,126,326,5,146,117,568,3.890411
"Judge, Aaron",2022,175,111,131,391,16,157,133,607,3.866242
"Trout, Mike",2019,120,110,104,303,11,134,110,518,3.865672


In [13]:
write_csv(fpoints_g, "data/fpoints_data.csv")